In [44]:
# coding: utf-8
import xml.etree.ElementTree as ETree
import pandas as pd
import json 

In [45]:
# Read xml
nodeData = '/Users/xinyun/Documents/data/Network.xml'
prstree = ETree.parse(nodeData)
root = prstree.getroot()

### **Get 2D list of node** 

In [46]:
# Create two lists
items = []
node = []

# Iterate through the root node to get the child node 
for n in root.iter('node') :
    id_nb = n.attrib.get('id')
    x = n.attrib.get('x')
    y = n.attrib.get('y')
    
    items = [id_nb, x, y]   # Create a list for each row
    node.append(items)

### **Get 2D list of link** 

In [47]:
# Create two lists
items_ = []
link = []
car = []
c = []
# Iterate through the root node to get the child node
for l in root.iter('link') :
    id_nb = l.attrib.get('id')
    start = l.attrib.get('from')
    end = l.attrib.get('to')
    length = l.attrib.get('length')
    modes = l.attrib.get('modes')
    items_ = [id_nb, start, end, length, modes]
    # List contains all kinds of modes of transport (car, pt)
    link.append(items_)
    # List contains only cars(car and pt,car)
    if (modes == 'pt,car') | (modes == 'car') :
        c = [id_nb, start, end, length, modes]
        car.append(c)      

### **Dictionary and Json file**

In [108]:
# Empty nested dictionary
Network = { 'ROADS': {'NODES': {}, 'STOPS': {}, 'SECTIONS': {} },
         'LAYERS': [{},{},{},{}]}
# Initialisation the first position of 'LY'
 Network['LAYERS'][0] = {'ID' : 'CAR', 'TYPE' : 'mnms.graph.layers.CarLayer',  'VEH_TYPE': 'mnms.vehicles.veh_type.Car',
    'DEFAULT_SPEED': 13.8,  'SERVICES': [], 'NODES': [], 'LINKS': [], 'MAP_ROADDB': {'NODES' : {}, 'LINKS' : {}}}

IndentationError: unexpected indent (2617970406.py, line 5)

In [106]:
for part,info in Network.items() :
    if part == 'ROADS' :
        for key in info :
            if key == 'NODES' :  # Nodes part
                for i in range(len(node)) :
                    id_node = node[i][0]
                    x = float(node[i][1])
                    y = float(node[i][2])
                    info[key][id_node] = [x,y]
            if key == 'SECTIONS' : # Sections part
                for i in range(len(link)) :
                    id_section = link[i][0]
                    upstream = link[i][1]
                    downstream = link[i][2]
                    length = float(link[i][3])
                    zone = 'RES'
                    info[key][id_section] = {'upstream' : upstream, 'downstream' : downstream, 'length' : length, 'zone' : zone}
               # Stops part ......
    if part == 'LAYERS' :
        # Car layer (first {})
        
        for key,value in  Network['LAYERS'][0].items() :
            if key == 'NODES' :
                for i in range(len(node)) :
                    ID = node[i][0]
                    X = float(node[i][1])
                    Y = float(node[i][2])
                    value.append({'ID' : ID, 'X' : X, 'Y' : Y, 'LABEL' : 'CAR', 'EXCLUDE_MOVEMENTS' : {}})
            if key == 'LINKS' :
                for i in range(len(car)) :
                    ID = car[i][0]
                    upstream = car[i][1]
                    downstream = car[i][2]
                    length = float(car[i][3])
                    value.append({'ID' : ID, 'UPSTREAM': upstream, 'DOWNSTREAM' : downstream, 'COSTS' : {'length' : length}, 'LABEL' : 'CAR'})
            if key == 'MAP_ROADDB' :
                for k,v in Network['LAYERS'][0]['MAP_ROADDB'].items() :
                    # Key = 'NODES'
                    if k == 'NODES' :
                        for i in range(len(node)) :
                            ID_node = node[i][0]  # Get node ID
                            v[ID_node] = ID_node
                    # Key = 'LINKS'
                    if k == 'LINKS' :
                        for i in range(len(car)) :
                            ID_link = car[i][0] # Get link ID
                            v[ID_link] = [ID_link]
        

In [103]:
# Serializing json  
json_object = json.dumps(Network, indent = 4) 

In [21]:
#print(json_object)

In [104]:
# Writing to network.json
with open("network.json", "w") as outfile:
    outfile.write(json_object)